# Task-2 Label data Subset of Dataset in CoNLL format

## 🔹 Imports and Setup

In [18]:
import csv
import importlib
import os
import re
import unicodedata
import nltk
from nltk.tokenize import word_tokenize
# from nltk.normalize import word_normalize
import pandas as pd
import sys
import importlib
import warnings
warnings.filterwarnings('ignore')
sys.path.append(' F:/Telegram_E-commerce-_extractor-/src')
import os
os.chdir('..')

## 🔹 Load  cleaned data 
Loads the cleaned and preprocessed CSV file containing Telegram data into a pandas DataFrame.

In [19]:
df = pd.read_csv('F:/Telegram_E-commerce-_extractor-/data/combined_processed.csv')

## 🔹 Import Custom Preprocessing Functions
Imports two custom functions used for text normalization and tokenization tailored for Amharic.

In [ ]:
import os
os.chdir('..')
from src.preprocessing_labelling import normalize_amharic_text, tokenize_amharic_text

def preprocess_amharic_text(text):
    if pd.isna(text):
        return []
    normalized_text = normalize_amharic_text(text)
    tokens = tokenize_amharic_text(normalized_text)
    print(tokens[:10])
    return tokens

In [8]:
df["Product Tokens"] = df["Product Description"].apply(preprocess_amharic_text)

['በቀላሉ', 'ከሰል', 'ለማያያዝ', 'የሚሆን', 'አነስ', 'ያለ', 'ቦታ', 'የማይዝ', 'በኤሌክትሪክ', 'የሚሰራ']
['በቀላሉ', 'ከሰል', 'ለማያያዝ', 'የሚሆን', 'አነስ', 'ያለ', 'ቦታ', 'የማይዝ', 'በኤሌክትሪክ', 'የሚሰራ']
['በቀላሉ', 'ከሰል', 'ለማያያዝ', 'የሚሆን', 'አነስ', 'ያለ', 'ቦታ', 'የማይዝ', 'በኤሌክትሪክ', 'የሚሰራ']
['ስድስት', 'አይነት', 'ቅርጽዋጋ', '700', 'ብርውስን', 'ፍሬ', 'ነው', 'ያለው', 'አድራሻመገናኛመሰረትደፋርሞልሁለተኛፎቅ', 'ቢሮ']
['2200ዋጋ', '2700', 'ብር', 'ውስን', 'ፍሬ', 'ነው', 'ያለው', 'አድራሻመገናኛመሰረትደፋርሞልሁለተኛፎቅ', 'ቢሮ', 'ቁ']
['3', 'በማንኛውም', 'ጠርሙስ', 'ጫፍ', 'የሚገጠም', 'ለዘይት', 'እና', 'መሰል', 'ነገሮች', 'መቀነሻ']
['3', 'በማንኛውም', 'ጠርሙስ', 'ጫፍ', 'የሚገጠም', 'ለዘይት', 'እና', 'መሰል', 'ነገሮች', 'መቀነሻ']
['1', 'ዋጋ', '400', 'ብርውስን', 'ፍሬ', 'ነው', 'ያለው', 'አድራሻመገናኛመሰረትደፋርሞልሁለተኛፎቅ', 'ቢሮ', 'ቁ']
['1', 'ዋጋ', '400', 'ብርውስን', 'ፍሬ', 'ነው', 'ያለው', 'አድራሻመገናኛመሰረትደፋርሞልሁለተኛፎቅ', 'ቢሮ', 'ቁ']
['በኤሌክትሪክየሚሰራ', 'ለቤት', 'መልካም', 'መዓዛን', 'የሚሰጥዋጋ', '1400', 'ብርውስን', 'ፍሬ', 'ነው', 'ያለን']
['በፈለጉት', 'አቅጣጫ', 'ልጅዎን', 'በምቾት', 'ማዘል', 'ያስችልዎታልዋጋ', '2400', 'ብርውስን', 'ፍሬ', 'ነው']
['በኤሌክትሪክ', 'የሚሰራ', 'ለቤትና', 'ለመኪና', 'መልካም', 'መዓዛን', 'የሚሰጥ', 'ዋጋ', '1100', 'ብርውስን']
['በኤሌክት

## 🔹 Define Keywords for Labelling

**Defines sets of keywords used to identify whether a token should be labeled as a product, location, or price entity for NER tagging.**

In [ ]:
product_keywords = {
    'ስማርትፎን', 'ጫማ', 'ቦርሳ', 'ላፕቶፕ', 'ቲሸርት', 'ቀሚስ', 'ሰዓት', 'ካሜራ',
    'ማዳመጫ', 'ብስክሌት', 'ልብስ', 'ማቀዝቀዣ', 'ቡታ', 'ልብስ', 'ጫማ', 'ጥፍር',
    'መጠጥ', 'ልጅ', 'መቀቀል', 'መገጣጠሚያ', 'ተብሌት', 'መጫወቻ', 'ምድጃ',
    'ሶፋ', 'መጽሐፍ', 'ሞባይል', 'ጠረጴዛ', 'ጃኬት', 'ሰንሰለት', 'ባትሪ'
}
location_keywords = {'አዲስ_አበባ', 'ቦሌ', 'ፒያሳ', 'መገናኛ', 'ሜክሲኮ',
                     'አራት_ኪሎ', 'ጎፋ', 'ለገሃር', 'ቦታ', 'ቢሮ', 'አድራሻ'}
price_keywords = {'በ', 'ዋጋ', 'ብር'}

## 🔹 Import and Apply Labeling Function
mports the label_tokens function that likely compares each token to the keyword sets and assigns BIO labels (e.g., B-PRODUCT, I-PRICE, or O).

In [20]:
## labelling 
import os
os.chdir('..')
from src.preprocessing_labelling import label_tokens
tokens1 = df['Product Tokens']
labeled_tokens = df['Product Tokens'].apply(label_tokens)

Stores original tokens and their corresponding labels for each row in preparation for exporting in CoNLL format.


In [21]:
coNLL_data=[]
for token, label in zip(tokens1, labeled_tokens):
    for t,l in zip(token,label):   
        coNLL_data.append(f"{t} {l}")
    coNLL_data.append("")
coNLL_data="\n".join(coNLL_data)

- Combines each token and its label into token label format.
- Inserts an empty line between sentences/messages to follow CoNLL standards.
- Joins all lines into a single string with newline separators.



In [24]:

# Define the directory and file path
output_dir = r"F:/Telegram_E-commerce-_extractor-/data/labeled"
output_file = "labeled_data.conll"
output_path = os.path.join(output_dir, output_file)

# Ensure the directory exists (create if it doesn't)
os.makedirs(output_dir, exist_ok=True)

# Check if coNLL_data is not empty
if not coNLL_data:
    print("Warning: coNLL_data is empty! File will be created but contain no data.")

try:
    # Write the file
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(coNLL_data)
    
    # Verify the file was saved
    if os.path.exists(output_path):
        print(f"Success! File saved to: {output_path}")
        print(f"File size: {os.path.getsize(output_path)} bytes")
    else:
        print("Error: File was not created.")

except Exception as e:
    print(f"An error occurred while saving the file: {e}")

Success! File saved to: F:/Telegram_E-commerce-_extractor-/data/labeled\labeled_data.conll
File size: 5312730 bytes


⚙️  Workflow Insight
my workflow is structured into 4 essential NER preprocessing stages:

Stage	What It Does
1. Data Loading	Reads cleaned product descriptions from a CSV
2. Tokenization & Normalization	Cleans Amharic text and splits into linguistically meaningful tokens
3. Rule-based Labeling	Uses predefined keyword dictionaries to tag tokens with BIO labels
4. CoNLL Export	Saves the token-label pairs in a format used to train/evaluate NER models

✅ The pipeline is modular and well-structured — easy to plug into model fine-tuning stages.

## 🧠 Insights from Current Results
Your labeled output uses BIO tagging conventions (B-PRICE, I-PRODUCT, O, etc.), which is the standard for CoNLL.
The preview you showed:

በ B-PRICE
ቀ O
ላ O
...
- confirms that the price keywords (like በ) are being detected and labeled correctly. But it also shows:
- ⚠️ Most tokens are labeled O — indicating limited keyword coverage.
- 🔎 This likely means:

    Many tokens in your product descriptions are not captured by the current keyword sets.

-   Some entities (e.g., brand names, place names not in the list) are missed.